# AI6121 Computer Vision Project 1: MNIST

In [ ]:
ver = 1
comments = ''

## Versioning, Changelogs & References

### Changelogs
+ V0.1 - Base codes with:
    + Config setting, seeding, checkpoint saving, criterion, optimizer, scheduler setting, dataset loading and model training

+ V0.2 - Fixed bug in ACC computation in utils/train_helper.py
  + Added LeNet5 in models/BasicCNN.py
  + Added code to train full training set with best hyper-parameters from cross validation 

+ V0.3 - Checkpoint Update
  + Added data_augmentation() function for easy augmentation tuning.
  + Moved checkpoints saving to be based on run_time for parallel training.

+ V0.4 - Visualization Update
  + Added AJCNN to create model options
  + Added TorchViz
  + Added Tensorboard extensions to Notebook

+ V0.5 - Augmentations Update
  + Added Torchvision Data Augmentations
  + Added Albumentations Data Augmentations:
    + ScaleShiftRotate
    + Distortion
    + ElasticTransform (Elastic Distortion) [Simard2003]

+ V0.6 - Error Analysis Update
  + Added Error Analysis for Testset.
    + Tabulate accuracy scores.
    + Plot mis-classified examples.

+ V1.0 - Cleanup for Submission


### References
+ [Official MNIST](https://yann.lecun.com/exdb/mnist/)
+ [PyTorch MNIST](https://pytorch.org/docs/stable/torchvision/datasets.html#mnist)

## Setup/ Configuration

### 3rd Party Installations

In [ ]:
!pip install torchsummary
!pip install torchviz
!pip install albumentations==0.5.1

### Google Colab Setup

In [ ]:
import sys, os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/gdrive')
    file_name = f'ai6121-cv-p1-mnist-v{ver}.ipynb'
    import subprocess
    path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    #path_to_file = f'/content/gdrive/My Drive/AI6121 - CV Project/ai6121-cv-p1-mnist-v{ver}.ipynb'
    path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    print(path_to_file)
    os.chdir(path_to_file)
    !pwd

### Imports

In [ ]:
import random
import time
import numpy as np
from pprint import pprint
from tqdm import tqdm
import shutil
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.optim as optim
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter

import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from PIL import Image

from torchsummary import summary
from torchviz import make_dot

import utils
from utils import ModelTimer, AverageMeter
import models
from dataset.MNIST import MNISTDataset
from IPython.core.debugger import set_trace

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# set the backend of matplotlib to the 'inline' backend
%matplotlib inline

In [ ]:
# Test imports
print(torch.__version__, "Cuda Avail: ", torch.cuda.is_available())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'torch device: {device}')

In [ ]:
import importlib
importlib.reload(utils)
importlib.reload(models)

### Configurations

In [ ]:
config = utils.load_config_from_file('config.ini')

In [ ]:
# Manually update configurations (string) 
config.set('CONSTANTS','manual_seed', '42')
config.set('CONSTANTS','evaluate', 'False')

In [ ]:
if False: # update/save to config file (True / False)
    utils.update_config_to_file('config.ini', config)

In [ ]:
pConfig = config['PATHS']
cConfig = config['CONSTANTS']
dConfig = config['DEFAULT']
trainCfg = config['TRAIN']
valCfg = config['VAL']
testCfg = config['TEST']

### Seeding

In [ ]:
# set random seed for reproducibility
def seed_everything(seed=None):
    if seed is None:
        seed = random.randint(1, 10000) # create random seed
        print(f'random seed used: {seed}')
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    if 'torch' in sys.modules:
        print(f"seeding torch modules")
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
    return seed
    
seed = seed_everything(seed=cConfig.getint('manual_seed'))

## Dataset, DataLoaders

In [ ]:
def data_augmentations(augments=None, albumentation=False):
    if not albumentation:
        normalize = transforms.Normalize((0.1307,), (0.3081,))
        data_transforms = {
            'train': transforms.Compose([
                transforms.ToTensor(),
                normalize
            ]),
            'valid': transforms.Compose([
                transforms.ToTensor(),
                normalize
            ]),
            'test': transforms.Compose([
                transforms.ToTensor(),
                normalize
            ])
        }
        if augments is not None:
            all_augs = []
                
            all_augs += [transforms.ToTensor(), normalize]

            if 'RandomRotation' in augments: 
                all_augs.append(transforms.RandomRotation(degrees=10,
                                                          resample=Image.BILINEAR))
            if 'RandomAffine' in augments: 
                all_augs.append(transforms.RandomAffine(degrees=10, 
                                                        resample=Image.BILINEAR))
            
            if 'RandomPerspective' in augments: 
                all_augs.append(transforms.RandomPerspective(distortion_scale=0.2, p=0.5, 
                                                             interpolation=Image.BILINEAR))

            if 'RandomResizedCrop' in augments: 
                all_augs.append(transforms.RandomResizedCrop(size=(28,28)))
                
            if 'RandomErasing' in augments: # only random erasing is after to tensor
                all_augs.append(transforms.RandomErasing(p=0.3, scale=(0.02, 0.33), 
                                                         ratio=(0.3, 3.3), value=0))
            if 'RandomCropAndResize' in augments:   
                all_augs.append(transforms.RandomApply(torch.nn.ModuleList(
                    [ transforms.RandomCrop(24), transforms.Resize(28)]), p=0.5))   

            data_transforms['train'] = transforms.Compose(all_augs)
    else:
        normalizeA = A.Normalize(mean=[0.1307,],std=[0.3081,])
        data_transforms = {
            'train': A.Compose([
                normalizeA,
                ToTensorV2()
            ]),
            'valid': A.Compose([
                normalizeA,
                ToTensorV2()
            ]),
            'test': A.Compose([
                normalizeA,
                ToTensorV2()
            ])
        }
        if augments is not None:
            all_augs = []
            if 'ShiftScaleRotate' in augments:
                all_augs.append(A.ShiftScaleRotate(shift_limit = 0.1,
                                                    scale_limit = 0.1,
                                                    rotate_limit = 20,
                                                    interpolation = cv2.INTER_LANCZOS4,
                                                    border_mode = cv2.BORDER_CONSTANT,
                                                    p = 1))
            if 'Distortion' in augments:
                all_augs.append(A.OneOf([A.OpticalDistortion(border_mode = cv2.BORDER_CONSTANT, p=1.0),
                                         A.GridDistortion(border_mode = cv2.BORDER_CONSTANT,p=1.0)],
                                        p=0.5))
            if 'ElasticTransform' in augments: 
                all_augs.append(A.ElasticTransform(alpha=8, sigma=3, alpha_affine=2, p=0.5))
                
            all_augs += [normalizeA, ToTensorV2()]
            data_transforms['train'] = A.Compose(all_augs)

    return data_transforms

In [ ]:
# trainCfg['augments'] = 'RandomRotation' #'RandomRotation,RandomCropAndResize'
augCfg = None
if not trainCfg['augments'] == 'None':
    augCfg = trainCfg['augments'].split(',')
print(augCfg)

In [ ]:
test_da = data_augmentations(augCfg)
print(test_da)

In [ ]:
def load_mnist_datasets(augments = None, verbose=True, visualize=None):
    data_transforms = data_augmentations(augments)
    print(data_transforms)

    mnist_datasets = {}
    dataloaders = {}
    if dConfig.getboolean('evaluate'): # Load test set only
        test_dataset = MNISTDataset(root=pConfig['datapath'], train=False, 
                                    download=True, transform=data_transforms['test'])

        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=testCfg.getint('batch'), 
                       num_workers=dConfig.getint('num_workers'), pin_memory=dConfig.getboolean('pin_memory'))
        
        mnist_datasets['test'] = test_dataset
        dataloaders['test'] = test_loader
        
    else:
        train_dataset = MNISTDataset(pConfig['datapath'], train=True, 
                                     download=True, transform=data_transforms['train'])
        if valCfg.getfloat('split') > 0:
            # split validation set from train set
            valid_dataset = MNISTDataset(root=pConfig['datapath'], train=True, 
                                          download=True, transform=data_transforms['valid'])
            
            num_train = len(train_dataset)
            indices = list(range(num_train))
            np.random.shuffle(indices) # shuffle indices
            split = int(np.floor(valCfg.getfloat('split') * num_train))
            
            train_indices, valid_indices = indices[split:], indices[:split]
            train_sampler = SubsetRandomSampler(train_indices)
            valid_sampler = SubsetRandomSampler(valid_indices)
            
            mnist_datasets['train'] = torch.utils.data.Subset(train_dataset, train_indices)
            mnist_datasets['valid'] = torch.utils.data.Subset(valid_dataset, valid_indices)
        else:
            # Use test set as validation set
            print(f"Using test set as validation set!")
            valid_dataset = MNISTDataset(root=pConfig['datapath'], train=False, 
                                   download=True, transform=data_transforms['valid'])
            
            train_sampler = None
            valid_sampler = None
            mnist_datasets['train'] = train_dataset
            mnist_datasets['valid'] = valid_dataset

        train_loader = torch.utils.data.DataLoader(train_dataset, 
                      batch_size=trainCfg.getint('batch'), sampler=train_sampler, 
                      num_workers=dConfig.getint('num_workers'), 
                      pin_memory=dConfig.getboolean('pin_memory'))

        valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                      batch_size=valCfg.getint('batch'), sampler=valid_sampler, 
                      num_workers=dConfig.getint('num_workers'), 
                      pin_memory=dConfig.getboolean('pin_memory'))

        dataloaders['train'] = train_loader
        dataloaders['valid'] = valid_loader
        
    if verbose:
        dataset_sizes = {x: len(mnist_datasets[x]) for x in mnist_datasets.keys()}
        print(f"Dataset sizes: {dataset_sizes}")

    if visualize is not None:
        sample_loader = torch.utils.data.DataLoader(mnist_datasets[visualize], 
                                                    batch_size=9, shuffle=True, 
                                                    num_workers=dConfig.getint('num_workers'), 
                                                    pin_memory=dConfig.getboolean('pin_memory'))
        for idx, (images, labels) in enumerate(sample_loader):
            X = images.numpy()
            utils.plot_images(X, labels)
            break

    return mnist_datasets, dataloaders

dConfig['evaluate'] = 'False'
mnist_datasets, dataloaders = load_mnist_datasets(augments = augCfg, visualize='train')

## Create Model

In [ ]:
model_names = sorted(name for name in models.__dict__
                     if callable(models.__dict__[name]))
print(f"Available Models: {model_names}")

In [ ]:
def create_model(arch, device, verbose=True, plot=False):
    print("=> creating model '{}'".format(arch))
    if arch.startswith('LeNet5'):

        # Original LeNet5 arch default params
        kwargs = {}
        
        # Original LeNet5 arch params with relu and max pool
        # kwargs = {'activation':'relu', 'pool':'max'}

        # LeNet5 arch with more filters
        # kwargs = {'kernel':3, 'pad':1, 
        #          'activation':'relu', 'pool':'max',
        #          'num_filter1':50, 'num_filter2':100,
        #          'linear1':3600}

        model = models.__dict__[arch](**kwargs)
    elif arch.startswith('VGG'):
        model = models.__dict__['VGG'](variant=arch, batch_norm=False)
    elif arch.startswith('AJCNN'):
        model = models.__dict__['AJCNN'](variant=arch)
    else:
        model = models.__dict__[arch]()
    model = model.to(device)
    
    if verbose:
        print(model)
        summary(model, (1,28,28))
    if plot:
        eg_input = torch.zeros((64, 1, 28, 28), 
                               dtype=torch.float, requires_grad=False).to(device)
        y = model(eg_input)
        make_dot(y, params=dict(list(model.named_parameters()))).render(arch, format="png")
    return model

model = create_model(dConfig['model'], device, verbose=True, plot=False)    

### Save Checkpoint

In [ ]:
def save_ckp(state, is_best, checkpoint_path, bestmodel_path):
    print(f"=> saving checkpoint '{checkpoint_path}'")
    torch.save(state, checkpoint_path)
    if is_best:
        print(f"=> saving best model '{bestmodel_path}'")
        # copy that checkpoint file to best path given, bestmodel_path
        shutil.copyfile(checkpoint_path, bestmodel_path)

### Load Checkpoint

In [ ]:
def load_ckp(checkpoint_path, device, optimizer=None):
    print(f"=> loading checkpoint '{checkpoint_path}'")
    checkpoint = torch.load(checkpoint_path)
    print(f"=> creating model '{checkpoint['arch']}'")
    model = create_model(checkpoint['arch'], device)
    model.load_state_dict(checkpoint['state_dict'])

    epoch = checkpoint['epoch']
    best_val_acc = checkpoint['best_val_acc']

    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer'])

    lr = checkpoint['lr']
    total_time = checkpoint['total_time']
    model_timer = ModelTimer(total_time)
    print(f"=> loaded checkpoint '{checkpoint_path}' (epoch {epoch})")
    print(f"=> checkpoint's best val '{best_val_acc}' ({model_timer})")
    return model, best_val_acc, total_time

## Criterion, Optimizer, Scheduler

In [ ]:
class SmoothCrossEntropyLoss(nn.Module):
    def __init__(self, smoothing=0.0):
        super(SmoothCrossEntropyLoss, self).__init__()
        self.smoothing = smoothing
    
    def forward(self, input, target):
        log_prob = F.log_softmax(input, dim=-1)
        weight = input.new_ones(input.size()) * \
            self.smoothing / (input.size(-1) - 1.)
        weight.scatter_(-1, target.unsqueeze(-1), (1. - self.smoothing))
        loss = (-weight * log_prob).sum(dim=-1).mean()
        return loss

In [ ]:
def get_criterion(loss_name, device):
    criterion = nn.CrossEntropyLoss().to(device)
    if loss_name.startswith('SmoothCrossEntropyLoss'):
        criterion = SmoothCrossEntropyLoss(smoothing=0.003).to(device)
    return criterion

# criterion = get_criterion(config['DEFAULT']['criterion'], device)
# criterion

In [ ]:
def get_optimizer(model, opt_name, config=config):
    print("=> initializing optimizer '{}'".format(opt_name))
    optimizer = None
    parameters = model.parameters()
    lr = config.getfloat('DEFAULT', 'lr')
    mom = config.getfloat('DEFAULT', 'momentum')
    wd = config.getfloat('DEFAULT', 'weight_decay')
    if opt_name == 'SGD':
        optimizer = optim.SGD(parameters, lr, momentum=mom, weight_decay=wd)
    elif opt_name == 'Adam':
        optimizer = optim.Adam(parameters, lr, weight_decay=wd)
    elif opt_name == 'AdamW':
        optimizer = optim.AdamW(parameters, lr, weight_decay=wd)

    return optimizer

# config['DEFAULT']['optimizer'] = 'SGD'
# config['DEFAULT']['lr'] = '0.001'
optimizer = get_optimizer(model, config['DEFAULT']['optimizer'])

In [ ]:
def get_scheduler(optimizer, sch_name, config=trainCfg):
    print("=> initializing scheduler '{}'".format(sch_name))
    
    scheduler = None # Manual
    if sch_name == 'StepLR':
        step = config.getint('step_size')
        sgamma = config.getfloat('step_gamma')
        scheduler = lr_scheduler.StepLR(optimizer, step_size=step, gamma=sgamma)
    elif sch_name == 'MultiStepLR':
        sgamma = config.getfloat('step_gamma')
        scheduler = lr_scheduler.MultiStepLR(optimizer,  milestones=[30,60], gamma=sgamma)
    elif sch_name == 'ReduceLROnPlateau':
        patience = config.getfloat('plateau_patience')
        plat_factor = config.getfloat('plateau_factor')
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min',
                                                  factor=plat_factor,
                                                  patience=patience)
    return scheduler    

scheduler = get_scheduler(optimizer, trainCfg['scheduler'], config=trainCfg)

## Train/ Validation Functions


In [ ]:
def train(train_loader, model, criterion, optimizer, device):
    model.train()
    
    batch_time = AverageMeter()
    losses = AverageMeter()
    corrects = AverageMeter()

    end = time.time()
    for i, (X, y) in enumerate(tqdm(train_loader)):
          X = X.to(device, non_blocking=True)
          y = y.to(device, non_blocking=True)

          X.requires_grad_()
          
          optimizer.zero_grad()
          outputs = model(X)
          loss = criterion(outputs, y)

          losses.update(loss.detach().item(), X.size(0))
          corrects.update(utils.get_error(outputs.detach(), y))

          # compute bp
          loss.backward()
          optimizer.step()

          # measure elapsed time
          batch_time.update(time.time() - end)
          end = time.time()
          
          if (i + 1)% 100 == 0:
              print_line = '[Train] ({batch}/{size}) Batch: {bt:.3f}s | Loss: {loss:.6f} | Acc: {acc: .3f}'.format(
                          batch=i + 1,
                          size=len(train_loader),
                          bt=batch_time.avg,
                          loss=losses.avg,
                          acc=corrects.avg*100)
              print(print_line)

    return losses, corrects

In [ ]:
def validate(valid_loader, model, criterion, device):
    model.eval()
    
    batch_time = AverageMeter()
    losses = AverageMeter()
    corrects = AverageMeter()
    i = 0
    with torch.no_grad():
        end = time.time()
        for i, (X, y) in enumerate(tqdm(valid_loader)):
              X = X.to(device, non_blocking=True)
              y = y.to(device, non_blocking=True)
              
              outputs = model(X)
              loss = criterion(outputs, y)

              losses.update(loss.detach().item(), X.size(0)) 
              corrects.update(utils.get_error(outputs.detach(), y))

              # measure elapsed time
              batch_time.update(time.time() - end)
              end = time.time()

    print_line = '[Test] ({batch}/{size}) Batch: {bt:.3f}s | Loss: {loss:.6f} | Acc: {acc: .3f}'.format(
                batch=i + 1,
                size=len(valid_loader),
                bt=batch_time.avg,
                loss=losses.avg,
                acc=corrects.avg*100)
    print(print_line)
    return losses, corrects

## Trainer

In [ ]:
def trainer(dataloaders, model, criterion, optimizer, device, run_name, config=config):
    # visualization
    writer = SummaryWriter(os.path.join(pConfig['tensorboard_dir'], run_name))
    model_timer = ModelTimer()
    scheduler = get_scheduler(optimizer, config['TRAIN']['scheduler'], config=config['TRAIN'])
    
    stagnant_val_loss_ctr = 0
    min_val_loss = 1.
    epochs = trainCfg.getint('epochs')
    best_val_acc = 0
    for epoch in range(epochs):
        model_timer.start_epoch_timer()
        
        lr = optimizer.param_groups[0]['lr']

        print('\nEpoch: [%d | %d] LR: %.16f' % (epoch + 1, epochs, lr))

        # train for one epoch
        train_losses, train_accs = train(dataloaders['train'], model, criterion, 
                                         optimizer, device)
        
        # evaluate on validation set
        val_losses, val_accs = validate(dataloaders['valid'], model, criterion, 
                                        device)
        
        if config['TRAIN']['scheduler'] == 'ReduceLROnPlateau':
            scheduler.step(val_losses.avg)
        else:
            scheduler.step()  

        # tensorboardX
        writer.add_scalar('learning rate', lr, epoch + 1)
        writer.add_scalars('loss', {'train loss': train_losses.avg, 
                                    'validation loss': val_losses.avg}, epoch + 1)
        writer.add_scalars('accuracy', {'train accuracy': train_accs.avg*100, 
                                        'validation accuracy': val_accs.avg*100}, epoch + 1)

        is_best = val_accs.avg > best_val_acc
        best_val_acc = max(val_accs.avg, best_val_acc)
        model_timer.stop_epoch_timer()
        save_ckp({
            'epoch': epoch + 1,
            'arch': model.name,
            'state_dict': model.state_dict(),
            'best_val_acc': best_val_acc*100,
            'opt_name': config["DEFAULT"]["optimizer"],
            'optimizer' : optimizer.state_dict(),
            'lr': lr,
            'total_time': model_timer.total_time,
            'scheduler': config["TRAIN"]["scheduler"],
            'criterion': config["DEFAULT"]["criterion"]
        }, is_best, pConfig['checkpoint_fname'], pConfig['bestmodel_fname'])
        
        if trainCfg.getboolean('early_stopping'):
            if is_best:
                stagnant_val_loss_ctr = 0
                min_val_loss = val_losses.avg
            elif val_losses.avg >= min_val_loss:
                stagnant_val_loss_ctr += 1
                if (epoch+1) > trainCfg.getint('es_min') and stagnant_val_loss_ctr >= trainCfg.getint('es_patience'): 
                    break
            else:
                stagnant_val_loss_ctr = 0
                min_val_loss = val_losses.avg

    print("Training completed!")
    writer.close()
    
    print(f'Best accuracy: {best_val_acc*100}')
    return model_timer

## Main Function

### Get Run Name

In [ ]:
def get_run_name_time(seed, model, criterion, optimizer, comments, config=config):
    try:
        if criterion.name:
            p_criterion = criterion.name
    except:
        p_criterion = 'CE'

    lr = optimizer.param_groups[0]['lr']
    wd = optimizer.param_groups[0]['weight_decay']
    p_optimizer = f'{str(optimizer).split("(")[0].strip()}'
    p_optimizer += f'_lr{lr}'

    tb = config.getint('TRAIN', 'batch')
    epochs = config.getint('TRAIN', 'epochs')
    vs = config.getfloat('VAL', 'split')
    vb = config.getint('VAL', 'batch')
    sch_name = config.get('TRAIN', 'scheduler')
    if comments:
        comments = "_" + comments

    p_scheduler = f'{sch_name}'
    
    run_name = f'{model.name}_{seed}_e{epochs}_' \
                + f'tb{tb}_vs{vs}_vb{vb}_' \
                + f'{p_criterion}_{p_optimizer}_' \
                + f'{p_scheduler}' \
                + f'{comments}' 
                
    run_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    print(run_name, run_time)
    return run_name, run_time

In [ ]:
def setup_train_directories(run_name, run_time):
    checkpt_dir = pConfig['checkpoint_dir'] 
    run_cpt_dir = os.path.join(checkpt_dir, run_name, run_time)
    if not os.path.exists(run_cpt_dir):
        os.makedirs(run_cpt_dir)
    pConfig['checkpoint_fname'] = os.path.join(run_cpt_dir, 'checkpoint.pth.tar')
    pConfig['bestmodel_fname'] = os.path.join(run_cpt_dir, 'model_best.pth.tar')
    pConfig['final_model_fname'] = os.path.join(run_cpt_dir, 'model_final.pth.tar') #trainfullmodel_fname

    utils.update_config_to_file(os.path.join(run_cpt_dir, 'config.ini'), config)

# setup_train_directories(run_name, run_time)

### Training Configurations

In [ ]:
# Training Configurations
dConfig['model'] = 'AJCNN8'
dConfig['evaluate'] = 'False'
valCfg['split'] = '0.2'

dConfig['optimizer'] = 'SGD'
dConfig['lr'] = '0.01'
dConfig['criterion'] = 'SmoothCrossEntropyLoss'

trainCfg['epochs'] = '60'
trainCfg['augments'] = 'None'

# trainCfg['es_min'] = '10'
trainCfg['scheduler'] = 'StepLR'

### Training Loop

In [ ]:
mnist_datasets, dataloaders = load_mnist_datasets(augments = augCfg, visualize=None)
model = create_model(dConfig['model'], device, verbose=True)
criterion = get_criterion(dConfig['criterion'], device)
# criterion = SmoothCrossEntropyLoss(smoothing=0.003)

print(f"=> Training model: {not dConfig.getboolean('evaluate')}")

if not dConfig.getboolean('evaluate'):
    optimizer = get_optimizer(model, config['DEFAULT']['optimizer'])
    run_name, run_time = get_run_name_time(seed, model, criterion, optimizer, comments)
    setup_train_directories(run_name, run_time)
    mtimer = trainer(dataloaders, model, criterion, optimizer, device, run_name)
    print(f"=> Model trained time: {mtimer}")
    

In [ ]:
dConfig['evaluate'] = 'True'
if dConfig.getboolean('evaluate'):
    mnist_datasets, dataloaders = load_mnist_datasets(visualize='test')
    model, best_val_acc, total_time = load_ckp(pConfig['bestmodel_fname'], device)
    val_losses, val_accs = validate(dataloaders['test'], model, criterion, device)

## Train model with full training data with best cross-val hyper-params

In [ ]:
# Post tuning configurations
dConfig['evaluate'] = 'False'
valCfg['split'] = '0' # Use full train set for training

In [ ]:
def trainer_final(dataloaders, model, criterion, optimizer, device, run_name, config=config):

    # load best hyper-parameters from best model
    print(f"=> loading best model hyper-parameters '{pConfig['bestmodel_fname']}'")
    checkpoint = torch.load(pConfig['bestmodel_fname'])

    # visualization
    model_timer = ModelTimer()
    scheduler = get_scheduler(optimizer, config['TRAIN']['scheduler'], config=config['TRAIN'])

    epochs = checkpoint['epoch'] 
    for epoch in range(epochs):
        model_timer.start_epoch_timer()
        
        lr = optimizer.param_groups[0]['lr']

        print('\nEpoch: [%d | %d] LR: %.16f' % (epoch + 1, epochs, lr))

        # train for one epoch
        train_losses, train_accs = train(dataloaders['train'], model, criterion, 
                                         optimizer, device)
        
        if config['TRAIN']['scheduler'] == 'ReduceLROnPlateau':
            # evaluate on test set
            val_losses, val_accs = validate(dataloaders['valid'], model, criterion, 
                                            device)
            scheduler.step(val_losses.avg)
        else:
            scheduler.step()  

        model_timer.stop_epoch_timer()

    is_best = False 
    save_ckp({
        'epoch': epoch + 1,
        'arch': model.name,
        'state_dict': model.state_dict(),
        'opt_name': config["DEFAULT"]["optimizer"],
        'optimizer' : optimizer.state_dict(),
        'lr': lr,
        'total_time': model_timer.total_time,
        'scheduler': config["TRAIN"]["scheduler"],
        'criterion': config["DEFAULT"]["criterion"],
        'best_val_acc': None,
    }, is_best, pConfig['final_model_fname'], None)
        

    print("Training completed!")
    print("Train full model saved!")
    return model_timer


In [ ]:
mnist_datasets, dataloaders = load_mnist_datasets(augments = augCfg, visualize=None)
model = create_model(dConfig['model'], device, verbose=False)
# criterion = get_criterion(config['DEFAULT']['criterion'], device)
print(f'criterion: {criterion}')

print(f"=> Training model: {not dConfig.getboolean('evaluate')}")

if not dConfig.getboolean('evaluate'):
    optimizer = get_optimizer(model, config['DEFAULT']['optimizer'])
    mtimer = trainer_final(dataloaders, model, criterion, optimizer, device, run_name)
    print(f"=> Model trained time: {mtimer}")

## Evaluate on Testset

In [ ]:
dConfig['evaluate'] = 'True'
# pConfig['final_model_fname'] = 'checkpoints/AJCNN8_42_e60_tb128_vs0.2_vb64_CE_SGD_lr0.01_StepLR/20201127_030058/model_final.pth.tar'
if dConfig.getboolean('evaluate'):
    mnist_datasets, dataloaders = load_mnist_datasets(visualize='test')
    # model, best_val_acc, total_time = load_ckp(pConfig['bestmodel_fname'], device)
    model, best_val_acc, total_time = load_ckp(pConfig['final_model_fname'], device)
    val_losses, val_accs = validate(dataloaders['test'], model, criterion, device)

In [ ]:
def test_final(test_loader, model):
    y_preds = []
    y_labels = []

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (X, y) in enumerate(tqdm(test_loader)):
            # Overlapping transfer if pinned memory
            X = X.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            
            # compute output
            output = model(X)

            predicted_labels = output.argmax(dim=1)
            y_preds.extend(predicted_labels.detach().cpu().numpy())
            y_labels.extend(y.detach().cpu().numpy())
    
    preds_df = pd.DataFrame({
        'y_pred': y_preds,
        'y_label': y_labels,
    })
    return preds_df

def print_numbers_acc(accs, numbers=range(10)):
    assert len(accs) == len(numbers)
    assert type(accs[0]) == AverageMeter
    for t, a in zip(accs, numbers):
        print(f"{a}: {t.avg}")
    return {a:t.avg.item() for t, a in zip(accs, numbers)}

preds_df = test_final(dataloaders['test'], model)
preds_df

In [ ]:
corr_preds = preds_df[preds_df['y_pred'] == preds_df['y_label']]
corr_count = dict(zip(range(10),list(corr_preds['y_label'].value_counts().sort_index())))
for k,v in corr_count.items():
    corr_count[k] = v/preds_df[preds_df['y_label'] == k]['y_label'].count() * 100

corr_count 

In [ ]:
error_preds = preds_df[preds_df['y_pred'] != preds_df['y_label']]
error_preds.transpose()

In [ ]:
fig, ax = plt.subplots()
pd.value_counts(error_preds['y_label']).sort_index().plot(
    title='Model Error Count (by digit)',
    kind='bar', figsize=(10, 5), ax=ax, color='darkred', xlabel="digit", ylabel="count")

for p in ax.patches:
    value = round(p.get_height(),2)
    ax.annotate(str(value), xy=(p.get_x()+0.2, p.get_height()))

In [ ]:
def plot_sample_images(X, y, label, images_to_show=10, random=True):
    fig = plt.figure(1)
    # Set the canvas based on the numer of images
    fig.set_size_inches(18.5, images_to_show * 0.3)

    images_to_show = min(len(X), images_to_show)

    # Generate random integers (non repeating)
    if random == True:
        idx = np.random.choice(len(X), images_to_show, replace=False)
    else:
        idx = np.arange(images_to_show)
        
    # Print the images with labels
    for i in range(images_to_show):
        plt.subplot(images_to_show/10 + 1, 10, i+1)
        plt.title(f"Predict: {str(y[idx[i]])}\n GT: {label}")
        plt.imshow(X[idx[i]].reshape(28,28), cmap='gray')
        plt.axis('off') 

In [ ]:
error_preds_by_labels = dict.fromkeys(error_preds['y_label'].sort_index())
for i in error_preds_by_labels.keys():
    error_preds_by_labels[i] = list(zip(error_preds.loc[error_preds['y_label'] == i].index, error_preds.loc[error_preds['y_label'] == i, 'y_pred']))

# print(error_preds_by_labels)
for label, errors in error_preds_by_labels.items():
    # print(label, errors)
    error_img_indices = [x[0] for x in errors]
    mislabels = [x[1] for x in errors]
    error_images = [mnist_datasets['test'][i][0] for i in error_img_indices]
    error_images = torch.stack(error_images)
    # print(error_images)
    plot_sample_images(error_images, mislabels, label, 10, random=False)
    plt.savefig(f'error_analysis-{label}.png', bbox_inches='tight')
    plt.show()


# Tensorboard

In [ ]:
%tensorboard --logdir runs